In [10]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,max_error

def split_x_and_y(database,x,y):
  dataset_x = database[x]
  dataset_y = database[y]
  return dataset_x,dataset_y


def scores(Y_true, Y_predicted):
  r2 = r2_score(Y_true, Y_predicted)
  meansquarederror = mean_squared_error(Y_true, Y_predicted)
  meanabsoluteerror = mean_absolute_error(Y_true, Y_predicted)
  maxerror = max_error(Y_true, Y_predicted)

  print('r2:',r2,'meansquarederror:',meansquarederror,'meanabsoluteerror:',meanabsoluteerror,'maxerror:',maxerror)
  return r2,meansquarederror,meanabsoluteerror,maxerror

# TMAX

In [11]:
import pandas as pd
database_pre = pd.read_excel('database_TCC.xlsx')
database = database_pre[['A','E','N','Tmax']]
database

,A,E,N,Tmax
0,25,1,11,67.67
1,25,1,5,126.85
2,25,2,8,72.71
3,25,3,11,162.85
4,25,3,5,94.85
5,35,1,8,61.15
6,35,2,11,63.03
7,35,2,5,77.35
8,35,3,8,63.25
9,25,1,8,76.30


In [12]:
import pickle

with open("standard_scaler_Tmax.pkl", "rb") as f:
    rawdata = f.read()

standard_scaler_tmax = pickle.loads(rawdata)
data = standard_scaler_tmax.transform(database)
database = pd.DataFrame(data,columns=database.columns)

### Random Forest Regressor

In [15]:
with open("Random_Forest_Regressor_Tmax.pkl", "rb") as f:
    rawdata = f.read()
random_forest_regressor_tmax = pickle.loads(rawdata)

In [16]:
dataset_x,dataset_y = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Tmax')
test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Tmax')

random_forest_regressor_tmax.score(test_x,test_y)

0.705381030773407

In [17]:
random_forest_regressor_tmax.feature_importances_

array([0.21146886, 0.38204248, 0.40648866])

In [18]:
desnormalizar = test_x.copy()
desnormalizar['Tmax'] = random_forest_regressor_tmax.predict(test_x)
desnormalizado_teste = standard_scaler_tmax.inverse_transform(desnormalizar)
desnormalizado_teste

array([[25.        ,  1.        ,  8.        , 85.44523333],
       [25.        ,  2.        ,  5.        , 98.25708667],
       [35.        ,  3.        , 11.        , 95.42086667]])

In [19]:
random_forest_regressor_result = desnormalizado_teste[:,-1]

In [20]:
desnormalizar = test_x.copy()
desnormalizar['Tmax'] = test_y
desnormalizado_resultado = standard_scaler_tmax.inverse_transform(desnormalizar)
desnormalizado_resultado

array([[25.  ,  1.  ,  8.  , 76.3 ],
       [25.  ,  2.  ,  5.  , 98.76],
       [35.  ,  3.  , 11.  , 94.7 ]])

### Keras

In [22]:
from keras.models import load_model
keras_tmax = load_model('keras_model_Tmax.h5')

dataset_x,dataset_y = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Tmax')
test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Tmax')

In [23]:
desnormalizar = test_x.copy()
desnormalizar['Tmax'] = keras_tmax.predict(test_x.values)
desnormalizado_teste = standard_scaler_tmax.inverse_transform(desnormalizar)
desnormalizado_teste

1/1 [==============================] - 0s 164ms/step


array([[ 25.        ,   1.        ,   8.        ,  81.46868287],
       [ 25.        ,   2.        ,   5.        , 103.4628658 ],
       [ 35.        ,   3.        ,  11.        ,  93.59394363]])

In [24]:
keras_results = desnormalizado_teste[:,-1]

In [25]:
desnormalizar = test_x.copy()
desnormalizar['Tmax'] = test_y
desnormalizado_resultado = standard_scaler_tmax.inverse_transform(desnormalizar)
desnormalizado_resultado

answers = desnormalizado_resultado[:,-1]

In [26]:
scores(desnormalizado_resultado[:,-1],desnormalizado_teste[:,-1])

r2: 0.825284923165511 meansquarederror: 16.68519665631939 meanabsoluteerror: 3.659201678113812 maxerror: 5.168682866543691


(0.825284923165511, 16.68519665631939, 3.659201678113812, 5.168682866543691)

# RESULTS

In [27]:
df_results_tmax = pd.DataFrame(database_pre.iloc[[9,10,11]][['A','E','N','Tmax']])
df_results_tmax['Random Forest Regressor'] = random_forest_regressor_result
df_results_tmax['Keras'] = keras_results

In [28]:
df_results_tmax

,A,E,N,Tmax,Random Forest Regressor,Keras
9,25,1,8,76.30,85.445233,81.468683
10,25,2,5,98.76,98.257087,103.462866
11,35,3,11,94.70,95.420867,93.593944


In [29]:
df_results_tmax['average'] = df_results_tmax[['Random Forest Regressor','Keras']].mean(axis=1)

In [30]:
df_results_tmax

,A,E,N,Tmax,Random Forest Regressor,Keras,average
9,25,1,8,76.30,85.445233,81.468683,83.456958
10,25,2,5,98.76,98.257087,103.462866,100.859976
11,35,3,11,94.70,95.420867,93.593944,94.507405


In [31]:
scores(df_results_tmax['Tmax'],df_results_tmax['average'])

r2: 0.8056916122602655 meansquarederror: 18.556347398004522 meanabsoluteerror: 3.149843061279133 maxerror: 7.156958099938549


(0.8056916122602655, 18.556347398004522, 3.149843061279133, 7.156958099938549)